In [10]:
# Pandas is used for data manipulation
import pandas as pd
import numpy as np
from tpot import TPOTClassifier

# Read in data and display first 5 rows
z_data = pd.read_csv('esem_act_deact_zscore_groups.csv',sep=';',decimal='.')
z_data.drop(list(z_data.filter(regex = '_act')),axis = 1, inplace = True)

z_data = z_data.drop('scan', axis = 1)
z_data = z_data.drop('trial', axis = 1)
z_data = z_data.drop('response', axis = 1)

numLabels = 3 # num of different labels (comprehension and rest)
sizeTrainSet = 13 # num of participants used for training
sizeTestSet = 4 # num participants used for testing
numParticipants = sizeTrainSet + sizeTestSet

In [11]:
#group the data and get the labels after the grouping
#return the grouped data and the labels
def prepare(data,groupingColumns):
    grouped = data.groupby(groupingColumns)
    groupedAgg = grouped.aggregate(np.mean)
    labels = groupedAgg.index.get_level_values(level='task')

    return groupedAgg,labels

In [12]:
#split according to participants (first 12 and last 4)
#return all four sets (much like train_test_split from sklearn)
#TODO: introduce variation (see file coarseAverageParticipantSplit)

def split(features, labels, low, high):
    #first, drop everything non-numeric ===========================================================
    
    #=== creating training and validation set ======================================================
    # Saving feature names for later use
    feature_list = list(features.columns)

    # Convert to numpy array
    features = np.array(features)
    labels = np.array(labels)

    #get the rows excluding the last indexed (i.e., row with the index 587 is the last to include)
    training_features = features[0:low]
    training_features = np.array(training_features)
    testing_features = features[low:high]
    testing_features = np.array(testing_features)

    training_target = labels[0:low]
    training_target = np.array(training_target)
    testing_target = labels[low:high]
    testing_target = np.array(testing_target)
    #=== end creating training and validation set ===================================================
    
    return training_features, testing_features, training_target, testing_target

In [13]:
def findIndices(name):
    numGroups = int(name.split('_')[1])
    low = sizeTrainSet*numGroups*numLabels
    high = numParticipants*numGroups*numLabels
    return low, high

In [14]:
#define the columns to group by
groupingColumn1 = 'proband'
groupingColumn2 = 'task'
groupingColumn3 = ''

#define how many different variants of aggregation and aggregation levels
numVariationsPerAggLevel = 10
numAggLevels = 4
numAggregationVariants = numVariationsPerAggLevel * numAggLevels

#get the index of the starting column that contains the labels of the groupings
offset = z_data.columns.get_loc('aggr_2_groups0')

collectAccuracies = []

#collect all instances where unmatched is not possible.
unmatched = []

#loop through the columns
#for each column, group the data set
# transform it to numpy.ndarray and split into training and validation set
# run t_pot
# and do the learning
while offset < len(z_data.columns):
    # group the data set
    #=============================================================================================================
    columnsForGrouping = [groupingColumn1,groupingColumn2,z_data.columns[offset]]
    features, labels = prepare(z_data,columnsForGrouping)
    #=============================================================================================================

    #=============================================================================================================
    # transform to numpy.ndarray and do split
    # find low and high
    low, high = findIndices(z_data.columns[offset])
    training_features, testing_features, training_target, testing_target = split(features, labels, low, high)
    #=============================================================================================================
    
    folder = 'Deact/'
    fileName = 'tpot_mnist_pipeline_triangulateAggregationLevelParticipantSplit' + z_data.columns[offset] + '.py'
    #=============================================================================================================
    # run t_pot
    # TODO: extract method
    tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2,n_jobs=20)
    tpot.fit(training_features, training_target)
    print(tpot.score(testing_features, testing_target))
    tpot.export(folder + fileName)
    #=============================================================================================================    
        
    offset = offset + 1

Optimization Progress:  33%|███▎      | 40/120 [00:11<02:48,  2.10s/pipeline]

Generation 1 - Current best internal CV score: 0.6822222222222222


Optimization Progress:  50%|█████     | 60/120 [00:15<01:23,  1.39s/pipeline]

Generation 2 - Current best internal CV score: 0.6822222222222222


Optimization Progress:  67%|██████▋   | 80/120 [00:17<00:32,  1.22pipeline/s]

...

Optimization Progress:  67%|██████▋   | 80/120 [07:08<27:45, 41.63s/pipeline]  

Generation 3 - Current best internal CV score: 0.582051282051282


Optimization Progress:  83%|████████▎ | 100/120 [07:20<07:21, 22.10s/pipeline]

Generation 4 - Current best internal CV score: 0.582051282051282


Generation 5 - Current best internal CV score: 0.6000000000000001

Best pipeline: LogisticRegression(BernoulliNB(input_matrix, alpha=1.0, fit_prior=False), C=1.0, dual=False, penalty=l1)
0.6166666666666667


In [15]:
offset = z_data.columns.get_loc('aggr_2_groups0')

collectAccuracies = []

#collect all instances where unmatched is not possible.
unmatched = []

#loop through the columns
#for each column, group the data set
# transform it to numpy.ndarray and split into training and validation set
# run t_pot
# and do the learning
while offset < len(z_data.columns):
    # group the data set
    #=============================================================================================================
    columnsForGrouping = [groupingColumn1,groupingColumn2,z_data.columns[offset]]
    features, labels = prepare(z_data,columnsForGrouping)
    #=============================================================================================================

    #=============================================================================================================
    # transform to numpy.ndarray and do split
    # find low and high
    low, high = findIndices(z_data.columns[offset])
    training_features, testing_features, training_target, testing_target = split(features, labels, low, high)
    
    fileName = 'tpot_mnist_pipeline_triangulateAggregationLevelParticipantSplit' + z_data.columns[offset] + '.py'
    #=============================================================================================================
    #do the learning
    # TODO: extract method
    with open(folder + fileName) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    # content = [x.strip() for x in content] 

    #  or 'exported_pipeline = ' not in line
    cleanedContent = []
    for line in content:
        if 'tpot_data' not in line and 'training_target, testing_target' not in line:
            cleanedContent.append(line)
    #         content.remove(line)
    
    fileNameCleaned = folder + 'cleaned_' + fileName
    with open(fileNameCleaned, 'w') as filehandle:  
        for line in cleanedContent:
            filehandle.write('%s\n' % line)
            
    %run -i $fileNameCleaned
    
    # print the accuracy
    # TODO: extract method
    # TODO: print all accurracy values to one file
    num_matches = 0;
    for a, b in zip(testing_target, results):
        if a == b:
            num_matches = num_matches + 1
        else:
            unmatched.append((a,b))
    print('Number of matches:',num_matches,'(of',testing_target.size,')')

    accuracy = num_matches/testing_target.size*100
    print('Accuary: ',accuracy)
    #=============================================================================================================
    
    #=============================================================================================================
    #collect all accuracy values to plot them
    collectAccuracies.append(accuracy)

    offset = offset + 1

Number of matches: 18 (of 24 )
Accuary:  75.0


/users/author/ml_exec/venv1/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Number of matches: 12 (of 24 )
Accuary:  50.0
Number of matches: 19 (of 24 )
Accuary:  79.16666666666666
Number of matches: 12 (of 24 )
Accuary:  50.0


/users/author/ml_exec/venv1/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Number of matches: 18 (of 24 )
Accuary:  75.0
Number of matches: 13 (of 24 )
Accuary:  54.166666666666664
Number of matches: 16 (of 24 )
Accuary:  66.66666666666666
Number of matches: 18 (of 24 )
Accuary:  75.0
Number of matches: 16 (of 24 )
Accuary:  66.66666666666666
Number of matches: 15 (of 24 )
Accuary:  62.5
Number of matches: 25 (of 48 )
Accuary:  52.083333333333336
Number of matches: 22 (of 48 )
Accuary:  45.83333333333333
Number of matches: 31 (of 48 )
Accuary:  64.58333333333334
Number of matches: 23 (of 48 )
Accuary:  47.91666666666667
Number of matches: 28 (of 48 )
Accuary:  58.333333333333336
Number of matches: 38 (of 48 )
Accuary:  79.16666666666666
Number of matches: 27 (of 48 )
Accuary:  56.25
Number of matches: 22 (of 48 )
Accuary:  45.83333333333333
Number of matches: 27 (of 48 )
Accuary:  56.25
Number of matches: 26 (of 48 )
Accuary:  54.166666666666664


...

Number of matches: 38 (of 60 )
Accuary:  63.33333333333333
Number of matches: 33 (of 60 )
Accuary:  55.00000000000001
Number of matches: 39 (of 60 )
Accuary:  65.0
Number of matches: 37 (of 60 )
Accuary:  61.66666666666667
Number of matches: 35 (of 60 )
Accuary:  58.333333333333336
Number of matches: 39 (of 60 )
Accuary:  65.0
Number of matches: 35 (of 60 )
Accuary:  58.333333333333336
Number of matches: 42 (of 60 )
Accuary:  70.0
Number of matches: 27 (of 60 )
Accuary:  45.0
Number of matches: 37 (of 60 )
Accuary:  61.66666666666667


/users/author/ml_exec/venv1/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Number of matches: 66 (of 120 )
Accuary:  55.00000000000001
Number of matches: 57 (of 120 )
Accuary:  47.5
Number of matches: 67 (of 120 )
Accuary:  55.833333333333336
Number of matches: 75 (of 120 )
Accuary:  62.5


/users/author/ml_exec/venv1/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/users/author/ml_exec/venv1/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Number of matches: 67 (of 120 )
Accuary:  55.833333333333336
Number of matches: 76 (of 120 )
Accuary:  63.33333333333333
Number of matches: 61 (of 120 )
Accuary:  50.83333333333333
Number of matches: 60 (of 120 )
Accuary:  50.0
Number of matches: 70 (of 120 )
Accuary:  58.333333333333336
Number of matches: 74 (of 120 )
Accuary:  61.66666666666667


In [16]:
print(len(unmatched))
countComprehensionRest = 0
countComprehensionSyntax = 0
countRestSyntax = 0
for x in unmatched:
    if x == (0,1) or x == (1,0):
        countComprehensionRest = countComprehensionRest + 1
    if x == (0,2) or x == (2,0):
        countComprehensionSyntax = countComprehensionSyntax + 1
    if x == (1,2) or x == (2,1):
        countRestSyntax = countRestSyntax + 1
print('Comprehension/Rest',countComprehensionRest)
print('Comprehension/Syntax',countComprehensionSyntax)
print('Rest/Syntax',countRestSyntax)

1059
Comprehension/Rest 217
Comprehension/Syntax 444
Rest/Syntax 398


In [17]:
#input: alist: the list to split, wanted_parts: into how many parts it should be split
#returns: all split lists
def split_list(alist, wanted_parts=1):
    length = len(alist)
    return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]


In [18]:
list1, list2, list3, list4 = split_list(collectAccuracies,numAggLevels)
frames = [list1, list2, list3, list4]
collectedAccuracies = pd.DataFrame(frames).T

collectedAccuracies.columns = ['2_groups_deact','4_groups_deact','5_groups_deact','10_groups_deact']

collectedAccuracies.to_csv('triangulate_Deact.csv',sep=';',decimal='.', index=False)